# 调参

In [1]:
from hyperopt import fmin, tpe, Trials, hp,STATUS_OK
import numpy as np
import pandas as pd
import lightgbm as lgb
import os
import time
import csv

In [2]:
#读取训练集
train=pd.read_csv('data/train1.csv')

In [3]:
features=list(train.columns)
features.remove('flag')
X_train, y_train=train[features],train['flag']
##数据转换
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)

In [4]:
#%%参数空间
space = {
    # 'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt'}]),
    'num_leaves': hp.quniform('num_leaves', 10, 200, 5),
    'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
    'min_child_samples': hp.quniform('min_child_samples', 40, 401, 20),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 10),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 10),
    'subsample_freq':hp.uniform("subsample", 0.5, 0.9),
    'colsample_bytree':hp.quniform('colsample_bytree',0.7, 1.0, 0.1),
    'bagging_fraction':hp.quniform('bagging_fraction', 0.7, 1.0, 0.1),
    'max_depth': hp.randint("max_depth", 8),
    'objective': hp.choice('objective', ['binary']), 
    'n_estimators': hp.quniform('n_estimators',500,5000,200)
}


In [5]:
#%%目标函数
def objective(params, n_flods=5):  
    params['boosting_type'] = params['boosting_type']['boosting_type']
    for param in ['n_estimators','max_depth','num_leaves','min_child_samples','subsample_freq']:
        params[param] = int( params[param])
    
    start = time.clock()
    
    cv_results = lgb.cv(params=params,train_set=lgb_train, nfold=n_flods,
                        early_stopping_rounds=100, metrics='auc', seed=50,verbose_eval=False)
# =============================================================================
#     model_lgb = lgb.LGBMClassifier(boosting_type=params['boosting_type'],
#                     objective = params['objective'],
#                     metric = 'auc', 
#                     subsample_freq=params['subsample_freq'],
#                     learning_rate= params['learning_rate'],
#                     n_estimators = params['n_estimators'],
#                     max_depth = params['max_depth'],
#                     bagging_fraction = params['bagging_fraction'],
#                     colsample_bytree = params['colsample_bytree'],
#                     min_child_samples = params['min_child_samples'],
#                     reg_alpha = params['reg_alpha'],
#                     reg_lambda = params['reg_lambda'],
#                     seed=5000,
#                     n_jobs=4)
# =============================================================================
    best_score = np.max(cv_results['auc-mean'])
    loss = 1 - best_score
    
    run_time = (time.clock() - start)
    
    
    # 最大平均auc对应的提升术的迭代次数
    n_estimators = int(params['n_estimators'])
    
    # 把关心的结果输出到csv文件
    of_connection = open('result_lgb.csv', 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss,params,n_estimators,run_time])
    
    return_dict = {'loss':loss, 'params':params,'estimators':n_estimators,'train_time':run_time,'status':STATUS_OK}
    return return_dict


In [6]:
#%%优化算法algo
tpe_algorithm = tpe.suggest
bayes_trials = Trials()
best = fmin(fn = objective, space = space, algo = tpe_algorithm, 
                 max_evals = 100, trials = bayes_trials, rstate = np.random.RandomState(50),verbose=1)

columns=['loss', 'params', 'iteration', 'estimators', 'train_time']

best

  0%|                                    | 0/100 [00:00<?, ?it/s, best loss: ?]

c:\users\123\appdata\local\programs\python\python35\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



  1%|           | 1/100 [00:18<29:59, 18.18s/it, best loss: 0.0744022723877169]

c:\users\123\appdata\local\programs\python\python35\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



  2%|▏         | 2/100 [00:46<34:42, 21.25s/it, best loss: 0.07387033020904421]

...

c:\users\123\appdata\local\programs\python\python35\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



 99%|██████▉| 99/100 [1:19:42<00:39, 39.69s/it, best loss: 0.07292900344464515]

c:\users\123\appdata\local\programs\python\python35\lib\site-packages\lightgbm\engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



100%|██████| 100/100 [1:21:03<00:00, 48.64s/it, best loss: 0.07292900344464515]


{'bagging_fraction': 0.8,
 'boosting_type': 0,
 'colsample_bytree': 0.7000000000000001,
 'learning_rate': 0.0314557433111514,
 'max_depth': 7,
 'min_child_samples': 120.0,
 'n_estimators': 1200.0,
 'num_leaves': 170.0,
 'objective': 0,
 'reg_alpha': 7.348811880509189,
 'reg_lambda': 9.92159011937742,
 'subsample': 0.7226186898190853}